In [15]:
# choose an input image from data/drill_aug/test/images/ and run the model
import cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

CLASSES = "abcdefghijklmnopqrstuvwxyz0123456789"
def yoloPostAndPlot(output, resized_image, conf_threshold=0.5, nms_threshold=0.4):
    output = output[0].numpy()
    output = output.T
    image_height, image_width = resized_image.shape[:2]

    boxes_xywh = output[..., :4] 
    scores = np.max(output[..., 4:], axis=1)
    classes = np.argmax(output[..., 4:], axis=1)

    indices = cv2.dnn.NMSBoxes(boxes_xywh, scores, conf_threshold, nms_threshold)

    threshold = 0.25
    image_with_box = resized_image.copy()

    for i in indices:
        if scores[i] >= threshold:
            x_center, y_center, width, height = boxes_xywh[i]
            
            x1 = int((x_center - width / 2) * image_width)
            y1 = int((y_center - height / 2) * image_height)
            x2 = int((x_center + width / 2) * image_width)
            y2 = int((y_center + height / 2) * image_height)

            image_with_box = cv2.rectangle(image_with_box, (x1,y1), (x2,y2), [0,0,255], 1)
            cv2.putText(image_with_box, f'{CLASSES[classes[i]]}:{scores[i]:.2f}',(x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    return image_with_box
        
        

in_image = np.random.choice(glob('data/drill_aug/test/images/*.jpg'))
im_image = cv2.imread(in_image)
cv2.imshow('image', im_image)

# Load the saved_model in tensorflow
import tensorflow as tf
model = tf.saved_model.load('models/saved_model')

# Preprocess the image + convert to float tensor
im_image = cv2.resize(im_image, (1024, 1024)).astype(np.float32)
im_image = im_image / 255.0
im_image = np.expand_dims(im_image, axis=0)


# Run the model
out = model(im_image)

# Postprocess the output
# out = out.numpy()
# boxes, confidences, class_ids = yoloPostProcess(out, im_image.shape)

# Draw the boxes
# for box, conf, class_id in zip(boxes, confidences, class_ids):
#     left, top, width, height = box
#     cv2.rectangle(im_image, (left, top), (left+width, top+height), (0, 255, 0), 2)
#     cv2.putText(im_image, str(conf), (left, top), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

# out.shape
cv2.imshow(yoloPostAndPlot(out, im_image))

ImportError: libtiff.so.5: cannot open shared object file: No such file or directory